# Velib

## Config

In [1]:
API_KEY = "a44bff4661b1435a625ea7b661a3f6410386bcf3"
BOOTSTRAP_SERVERS = 'localhost:9092'
# For several servers
#BOOTSTRAP_SERVERS = ['localhost:9092', 'localhost:9093']

## Install package `kafka-python`

In [2]:
import sys
!conda install --yes --prefix {sys.prefix} kafka-python

# Or via pip
#import sys
#!{sys.executable} -m pip install kafka-python

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.6.2

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



## Kafka Producer (velib-get-stations.py)

In [ ]:
import json
import time
import urllib.request

from kafka import KafkaProducer

#API_KEY = "XXX" # FIXME Set your own API key here
url = "https://api.jcdecaux.com/vls/v1/stations?apiKey={}".format(API_KEY)

producer = KafkaProducer(bootstrap_servers=BOOTSTRAP_SERVERS)

while True:
    response = urllib.request.urlopen(url)
    stations = json.loads(response.read().decode())
    for station in stations:
        producer.send("velib-stations", json.dumps(station).encode(),
                key=str(station["number"]).encode()) # dispatch each stations always on same partition
    print("{} Produced {} station records".format(time.time(), len(stations)))
    time.sleep(1)

## Kafka Broker

In [ ]:
%%bash
./bin/zookeeper-server-start.sh ./config/zookeeper.properties
./bin/kafka-server-start.sh ./config/server.properties
./bin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic velib-stations

## Kafka Consumer (velib-monitor-stations.py)

In [3]:
import json
from kafka import KafkaConsumer

stations = {}
consumer = KafkaConsumer("velib-stations", bootstrap_servers='kafka:9092', group_id="velib-monitor-stations")

for message in consumer:
    station = json.loads(message.value.decode())
    station_number = station["number"]
    contract = station["contract_name"]
    available_bike_stands = station["available_bike_stands"]

    if contract not in stations:
        stations[contract] = {}

    city_stations = stations[contract]

    if station_number not in city_stations:
        city_stations[station_number] = available_bike_stands

    count_diff = available_bike_stands - city_stations[station_number]

    if count_diff != 0:
        city_stations[station_number] = available_bike_stands
        print("{}{} {} ({})".format(
            "+" if count_diff > 0 else "",
            count_diff, station["address"], contract
        ))

KeyboardInterrupt: 

## Modification de la durée de conservation des messages dans Kafka

Suppression d'un segment toutes les 4 secondes

In [ ]:
! ./bin/kafka-configs.sh --zookeeper localhost:2181 --entity-type topics --entity-name velib-stations --alter --add-config retention.ms=4000

Modification de la longueur d'un segment

In [ ]:
! ./bin/kafka-configs.sh --zookeeper localhost:2181 --entity-type topics --entity-name velib-stations --alter --add-config segment.ms=2000